# 영화 추천 시스템
## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python',encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 평점과 제목 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


## 2. 데이터 분석

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# ratings에 있는 유니크한 영화 제목 수
ratings['title'].nunique()

3628

In [10]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3. 내가 선호하는 영화 추가

In [13]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('star wars', regex=False)]

,movie_id,title,genre
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi


In [14]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [506, 329, 118 , 1203, 777]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'HajinJ'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['HajinJ']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,HajinJ,506,5,Orlando (1993)
1,HajinJ,329,5,Star Trek: Generations (1994)
2,HajinJ,118,5,If Lucy Fell (1996)
3,HajinJ,1203,5,12 Angry Men (1957)
4,HajinJ,777,5,Pharaoh's Army (1995)


In [15]:
if not ratings.isin({'user_id':['HajinJ']})['user_id'].any():  # user_id에 'HajinJ'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,HajinJ,506,5,Orlando (1993)
836479,HajinJ,329,5,Star Trek: Generations (1994)
836480,HajinJ,118,5,If Lucy Fell (1996)
836481,HajinJ,1203,5,12 Angry Men (1957)
836482,HajinJ,777,5,Pharaoh's Army (1995)


## 4. indexing 후 CSR matrix 생성

In [16]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,506,5,215
836479,6039,329,5,558
836480,6039,118,5,1264
836481,6039,1203,5,653


In [18]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 훈련시키기

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악

In [23]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
HajinJ, star_trek = user_to_idx['HajinJ'], movie_to_idx['Star Trek: Generations (1994)']
HajinJ_vector, star_trek_vector = als_model.user_factors[HajinJ], als_model.item_factors[star_trek]

In [24]:
HajinJ_vector   # 나의 벡터

array([ 4.48896170e-01,  8.40168148e-02, -4.17887479e-01,  1.71655729e-01,
        5.00877528e-03,  2.40890440e-02,  3.15483630e-01,  6.51768669e-02,
        9.10709947e-02, -3.33983958e-01,  2.89366901e-01,  4.93275821e-01,
        4.43770617e-01,  1.07495859e-03,  4.38158989e-01,  2.86489636e-01,
       -2.89361417e-01,  2.93684870e-01, -1.91965029e-01,  1.38539793e-02,
       -2.65937388e-01,  5.85881472e-01, -2.99357265e-01,  2.34048128e-01,
       -3.46339524e-01,  2.54105568e-01,  1.81027092e-02,  3.74945551e-01,
       -4.76025701e-01, -4.64201033e-01, -6.51710015e-03,  2.25439802e-01,
       -3.35303694e-01,  7.84168720e-01,  2.72727132e-01, -4.43724960e-01,
        3.38013321e-01,  8.56443122e-02,  2.60660678e-01,  7.03890622e-02,
       -4.55014482e-02,  3.74829054e-01, -2.70751268e-01,  6.26667142e-02,
        6.82308972e-02, -4.43704247e-01,  2.39492908e-01, -1.37825325e-01,
       -1.79908901e-01,  1.48758397e-01,  5.92891201e-02, -1.25817537e-01,
       -3.04726034e-01, -

In [25]:
star_trek_vector   # 영화 스타트렉의 벡터

array([ 0.01120893,  0.00461602,  0.01980377, -0.00336151,  0.00817345,
        0.00920888,  0.01340752, -0.01062332,  0.0123034 ,  0.01284194,
        0.00968032,  0.00702211,  0.02545374,  0.02464304,  0.01955541,
        0.01288115, -0.01122258, -0.01213378, -0.01135531,  0.01081989,
       -0.00831717,  0.00063565, -0.00815744,  0.0228998 , -0.01940267,
        0.01719087,  0.01012972,  0.03071558, -0.008157  ,  0.00345673,
        0.01038576,  0.01276994, -0.00487184,  0.02299052,  0.00564004,
        0.01189412,  0.01791419,  0.00426435,  0.02565641,  0.01423572,
       -0.00757588,  0.05405318, -0.00626657,  0.00129921, -0.00188276,
       -0.02572905,  0.02687287,  0.01183492, -0.00595451,  0.00994767,
        0.01437533,  0.00746818, -0.01176859, -0.00571443,  0.0140321 ,
       -0.01601525,  0.00538859,  0.02285075, -0.00870795,  0.00214138,
        0.03730858,  0.00668663, -0.02258314,  0.03525269,  0.01263063,
        0.02291804, -0.00442981, -0.01388126, -0.01580526, -0.01

In [26]:
# 나와 스타트렉의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(HajinJ_vector, star_trek_vector)

0.33905944

In [27]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['Third Man, The (1949)']
matrix_vector = als_model.item_factors[matrix]
np.dot(HajinJ_vector, matrix_vector)

0.022918968

## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [28]:
favorite_movie = 'Star Wars: Episode IV - A New Hope (1977)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(44, 1.0000001),
 (117, 0.87169737),
 (64, 0.7604658),
 (120, 0.71265656),
 (193, 0.49412188),
 (60, 0.479984),
 (200, 0.4590344),
 (124, 0.44625583),
 (22, 0.43014985),
 (26, 0.42167404),
 (172, 0.3809474),
 (92, 0.37266546),
 (5, 0.36220098),
 (197, 0.3618748),
 (3499, 0.35315493)]

In [29]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Raiders of the Lost Ark (1981)',
 'Alien (1979)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Terminator, The (1984)',
 'Matrix, The (1999)',
 'Back to the Future (1985)',
 'E.T. the Extra-Terrestrial (1982)',
 'Indiana Jones and the Last Crusade (1989)',
 'Terminator 2: Judgment Day (1991)',
 'Princess Bride, The (1987)',
 'Jaws (1975)',
 'Simon Sez (1999)']

In [30]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [31]:
get_similar_movie('E.T. the Extra-Terrestrial (1982)')

['E.T. the Extra-Terrestrial (1982)',
 'Close Encounters of the Third Kind (1977)',
 'Back to the Future (1985)',
 'Dark Crystal, The (1982)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Big (1988)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Time Bandits (1981)',
 'Honey, I Shrunk the Kids (1989)',
 'Driving Miss Daisy (1989)']

In [32]:
get_similar_movie('Forrest Gump (1994)')

['Forrest Gump (1994)',
 'Groundhog Day (1993)',
 'Pretty Woman (1990)',
 'Ghost (1990)',
 'Sleepless in Seattle (1993)',
 'Four Weddings and a Funeral (1994)',
 'As Good As It Gets (1997)',
 'Notting Hill (1999)',
 'Clueless (1995)',
 "You've Got Mail (1998)"]

## 8. 내가 가장 좋아할 만한 영화 추천

In [33]:
user = user_to_idx['HajinJ']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(113, 0.31122503),
 (471, 0.29369858),
 (699, 0.2849601),
 (584, 0.23399968),
 (586, 0.20780495),
 (581, 0.18203303),
 (583, 0.1774892),
 (1164, 0.17277713),
 (47, 0.14742175),
 (1109, 0.14436877),
 (100, 0.14012533),
 (60, 0.1278432),
 (876, 0.12688604),
 (544, 0.12650177),
 (569, 0.121323965),
 (1283, 0.11643197),
 (1505, 0.111143604),
 (58, 0.1098975),
 (2325, 0.106759146),
 (630, 0.10560131)]

In [34]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Star Trek VI: The Undiscovered Country (1991)',
 'Star Trek: First Contact (1996)',
 'Star Trek: Insurrection (1998)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek IV: The Voyage Home (1986)',
 'Star Trek: The Motion Picture (1979)',
 'Star Trek: The Wrath of Khan (1982)',
 'Star Trek V: The Final Frontier (1989)',
 'To Kill a Mockingbird (1962)',
 'Vertigo (1958)',
 'Amadeus (1984)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 '2010 (1984)',
 'Stargate (1994)',
 'Rear Window (1954)',
 'Inherit the Wind (1960)',
 'Stalag 17 (1953)',
 'Mission: Impossible (1996)',
 'Anatomy of a Murder (1959)',
 'Mr. Smith Goes to Washington (1939)']

In [35]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Back to the Future (1985)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('12 Angry Men (1957)', 0.05942055730702799),
 ('Orlando (1993)', 0.003193649694773048),
 ("Pharaoh's Army (1995)", 0.0004475744231371138),
 ('If Lucy Fell (1996)', -0.0003257948523664235),
 ('Star Trek: Generations (1994)', -0.03739403022328118)]

***
# 프로젝트 정리
- **favorite 리스트에 넣었던 영화 'Star Wars'과의 벡터 내적은 비교적 높았고, 선호 리스트에 넣지 않은 영화 'Third Man'와의 벡터 내적은 비교적 낮았다. 내가 좋아하는 영화와 비슷한 영화 추천도 액션이면 액션, SF면 SF 등 유사한 장르의 영화가 추천되었다. 그리고 내가 좋아할 만한 영화 추천 목록에도 실제로 보고 싶은 영화들이 포함되어 있어 추천 모델의 성능이 꽤 괜찮은 것 같았다.